# Table Expressions

## Introduction

**Table expressions** in T-SQL are named **query expressions** that represent a table. They are <u>used to simplify complex queries</u> and improve readability.

They allow for efficient and flexible querying. They enable better modularity and reuse of queries.

There are four main types of **table expressions**:
1. Derived Tables.
2. Common Table EXpressions(CTEs).
3. Views.
4. Inline Table-Valued Functions (TVFs)

1. **Derived Tables**:
- These are _subqueries in the FROM clause_ that are given an alias.
- They exist only for the _duration of the query_.

    &emsp;&emsp;&emsp;&emsp;SELECT *<br>
    &emsp;&emsp;&emsp;&emsp;FROM `(SELECT column1, column2 FROM table1)` AS DerivedTable

    <hr>

2. **Commond Table Expressions _(CTEs)_**:
- These are _temporary result sets_ that can be referenced within a SELECT, INSERT, UPDATE, or DELETE statement.
- They are defined using the _WITH_ keyword.

&emsp;&emsp;&emsp;_WITH CTE AS_<br>
&emsp;&emsp;&emsp;`(`<br>
&emsp;&emsp;&emsp;&emsp;`SELECT column1, column2`<br>
&emsp;&emsp;&emsp;&emsp;`FROM table1`<br>
&emsp;&emsp;&emsp;`)`<br>
&emsp;&emsp;&emsp;_SELECT *_<br>
&emsp;&emsp;&emsp;_FROM CT_

<hr>

3. **Views**: 
- These are _virtual tables_ that are defined by a SELECT query.
- They are _stored in the database_ and can be queried like regular tables.

&emsp;&emsp;&emsp;&emsp;_`CREATE VIEW ViewName AS`_<br>
&emsp;&emsp;&emsp;&emsp;`SELECT column1, column2`<br>
&emsp;&emsp;&emsp;&emsp;`FROM table1`

<hr>

4. **Inline Table-Valued Functions _(TVFs)_**: 
- These are _functions_ that return a table. 
- They are defined using the _RETURNS TABLE_ clause.

&emsp;&emsp;&emsp;&emsp;_CREATE FUNCTION `FunctionName()`_<br>
&emsp;&emsp;&emsp;&emsp;_RETURNS TABLE_<br>
&emsp;&emsp;&emsp;&emsp;_AS_<br>
&emsp;&emsp;&emsp;&emsp;_RETURN_ `(`<br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;`SELECT column1, column2`<br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;`FROM table1`<br>
&emsp;&emsp;&emsp;&emsp;`)`

<hr>

#### Example of Derived Table

- Define a derived table called _USACusts_ based on a query that returns all customers from the United States, and the outer query selects all rows from the derived table.


In [ ]:
SELECT *
FROM (SELECT custid, companyname
      FROM Sales.Customers
      WHERE country = N'USA') AS USACusts

With all types of table expressions, a query must meet **_three requirements_** to be a <u>valid inner query</u> in a table expression definition:

- Order is not guaranteed.
- All columns must have names.
- All column names MUST be unique.

#### Assigning column aliases

One of the benefits of using table expressions is that, in any clause of the outer query, you can refer to 
column aliases that were assigned in the SELECT clause of the inner query. 

In [ ]:
-- Error: 

/*
The following attempt is invalid because the GROUP BY clause refers 
to a column alias that was assigned in the SELECT clause, and the GROUP BY
clause is logically processed prior to the SELECT clause
*/

SELECT
  YEAR(orderdate) AS orderyear,
  COUNT(DISTINCT custid) AS numcusts
FROM Sales.Orders
GROUP BY orderyear;

-- One solution: YEAR(orderdate) in both the GROUP BY and SELECT clauses

In [ ]:
-- To avoid the repetition of the code

SELECT orderyear, COUNT(DISTINCT custid) AS numcusts
FROM (SELECT YEAR(orderdate) AS orderyear, custid
      FROM Sales.Orders) AS D
GROUP BY orderyear;

It uses the inline aliasing form to assign column aliases to expressions.

The syntax for inline aliasing is `<expression> [AS] <alias>`. 

- Note that the word `AS` is optional in the syntax for inline aliasing; however, it helps the readability of the code and **recommend using it**.


In [ ]:
SELECT orderyear, COUNT(DISTINCT custid) AS numcusts
FROM (SELECT YEAR(orderdate), custid
      FROM Sales.Orders) AS D(orderyear, custid)
GROUP BY orderyear;

#### Using arguments

- Local Variables
- Input parameters to a routine (stored procedure or function)

In [ ]:
DECLARE @empid AS INT = 3;

SELECT orderyear, COUNT(DISTINCT custid) AS numcusts
FROM (SELECT YEAR(orderdate) AS orderyear, custid
      FROM Sales.Orders
      WHERE empid = @empid) AS D
GROUP BY orderyear;

#### Nesting

- The inner query (subquery) is executed first.

Be careful: Nesting tends to complicate the code and reduces its readability.

In [ ]:
-- Nested derived tables

SELECT orderyear, numcusts
FROM (SELECT orderyear, COUNT(DISTINCT custid) AS numcusts
    FROM (SELECT YEAR(orderdate) AS orderyear, custid
        FROM Sales.Orders) AS D1
    GROUP BY orderyear) AS D2
WHERE numcusts > 70;

#### Multiple references

The fact that you cannot refer to multiple instances of the same derived table in the same join forces you to maintain multiple copies of the same query definition. This leads to lengthy code that is hard to maintain and prone to errors.

In [ ]:
SELECT  Cur.orderyear,
        Cur.numcusts AS curnumcusts, Prv.numcusts AS prvnumcusts,
        Cur.numcusts - Prv.numcusts AS growth
FROM (SELECT YEAR(orderdate) AS orderyear,
             COUNT(DISTINCT custid) AS numcusts
      FROM Sales.Orders
      GROUP BY YEAR(orderdate)) AS Cur
LEFT OUTER JOIN
     (SELECT YEAR(orderdate) AS orderyear,
             COUNT(DISTINCT custid) AS numcusts
      FROM Sales.Orders
      GROUP BY YEAR(orderdate)) AS Prv
ON Cur.orderyear = Prv.orderyear + 1;

#### Common Table Expressions (CTEs)

- Are defined by using a `WITH` statement.

In [ ]:
/* SYNTAX:

WITH <CTE_Name>[(<target_column_list>)]
AS
(
  <inner_query_defining_CTE>
)
<outer_query_against_CTE>;

*/

The following code defines a **CTE** called _USACusts_ based on a query that returns all customers from the United States, and the outer query selects all rows from the **CTE**:

In [ ]:
WITH USACusts AS
(
  SELECT custid, companyname
  FROM Sales.Customers
  WHERE country = N'USA'
)
SELECT * FROM USACusts;

#### Assigning Column Aliases in CTEs

- CTEs also support two forms of column aliasing: _inline_ and _external_.

- <u>Inline</u>: `<expression> AS <column_alias>;`

In [ ]:
-- Inline

WITH C AS
(
  SELECT YEAR(orderdate) AS orderyear, custid
  FROM Sales.Orders
)
SELECT orderyear, COUNT(DISTINCT custid) AS numcusts
FROM C
GROUP BY orderyear;

- <u>External</u>:  specifying the target column list in parentheses immediately after the CTE name.

In [ ]:
-- External

WITH C(orderyear, custid) AS
(
  SELECT YEAR(orderdate), custid
  FROM Sales.Orders
)
SELECT orderyear, COUNT(DISTINCT custid) AS numcusts
FROM C
GROUP BY orderyear;

#### Using Arguments in CTEs

- CTEs can reference variables or arguments passed to the query.

In [ ]:
DECLARE @empid AS INT = 3;
WITH C AS
(
  SELECT YEAR(orderdate) AS orderyear, custid
  FROM Sales.Orders
  WHERE empid = @empid
)
SELECT orderyear, COUNT(DISTINCT custid) AS numcusts
FROM C
GROUP BY orderyear;

#### Defining Multiple CTEs

- You can define more than one CTE in a query, separated by commas.

In [ ]:
WITH C1 AS
(
  SELECT YEAR(orderdate) AS orderyear, custid
  FROM Sales.Orders
),
    C2 AS
(
  SELECT orderyear, COUNT(DISTINCT custid) AS numcusts
  FROM C1
  GROUP BY orderyear
)
SELECT orderyear, numcusts
FROM C2
WHERE numcusts > 70;

#### Multiple References in CTEs

- You can refer to a CTE multiple times in the query.

In the following code, the CTE _YearlyCount_ is defined <u>only once</u> and <u>accessed twice</u> in the FROM clause of the outer query—once as _Cur_ and once as _Prv_.

In [ ]:
WITH YearlyCount AS
(
  SELECT YEAR(orderdate) AS orderyear,
    COUNT(DISTINCT custid) AS numcusts
  FROM Sales.Orders
  GROUP BY YEAR(orderdate)
)
SELECT Cur.orderyear,
  Cur.numcusts AS curnumcusts, Prv.numcusts AS prvnumcusts,
  Cur.numcusts - Prv.numcusts AS growth
FROM YearlyCount AS Cur
  LEFT OUTER JOIN YearlyCount AS Prv
    ON Cur.orderyear = Prv.orderyear + 1;

#### Recursive CTEs

- A recursive CTE allows you _to perform hierarchical or recursive_ queries.
- It uses a <u>base</u> query and a <u>recursive</u> query that references the CTE itself.

In [ ]:
/* SYNTAX:

WITH <CTE_Name>[(<target_column_list>)]
AS
(
  <anchor_member>
  UNION ALL
  <recursive_member>
)
<outer_query_against_CTE>;

*/

- The recursive member joins the CTE—representing the previous result set—with the _Employees_ table to **return the direct subordinates** of the employees returned in the previous result set:

In [ ]:
WITH EmpsCTE AS
(
  SELECT empid, mgrid, firstname, lastname
  FROM HR.Employees
  WHERE empid = 2
  
  UNION ALL
  
  SELECT C.empid, C.mgrid, C.firstname, C.lastname
  FROM EmpsCTE AS P
    INNER JOIN HR.Employees AS C
      ON C.mgrid = P.empid
)
SELECT empid, mgrid, firstname, lastname
FROM EmpsCTE;

#### Views

- A view is a stored query that you can treat as a table.

In [ ]:
CREATE OR ALTER VIEW Sales.USACusts
AS
SELECT
  custid, companyname, contactname, contacttitle, address,
  city, region, postalcode, country, phone, fax
FROM Sales.Customers
WHERE country = N'USA';
GO

-- DROP VIEW IF EXISTS Sales.USACusts;
-- GO

#### Views and the ORDER BY Clause

- You cannot use `ORDER BY` in a view directly, unless it is accompanied by `TOP` or `OFFSET-FETCH`.

In [ ]:
-- Error

CREATE OR ALTER VIEW Sales.USACusts
AS
SELECT
  custid, companyname, contactname, contacttitle, address,
  city, region, postalcode, country, phone, fax
FROM Sales.Customers
WHERE country = N'USA'
ORDER BY region;
GO

-- DROP VIEW IF EXISTS Sales.USACusts;
-- GO

In [ ]:
-- Using TOP clause

CREATE OR ALTER VIEW Sales.USACusts
AS
SELECT TOP (100) PERCENT
  custid, companyname, contactname, contacttitle, address,
  city, region, postalcode, country, phone, fax
FROM Sales.Customers
WHERE country = N'USA'
ORDER BY region;
GO

-- SELECT custid, companyname, region
-- FROM Sales.USACusts;

In [ ]:
-- OFFSET clause with 0 ROWS, and without a FETCH clause.

CREATE OR ALTER VIEW Sales.USACusts
AS
SELECT
  custid, companyname, contactname, contacttitle, address,
  city, region, postalcode, country, phone, fax
FROM Sales.Customers
WHERE country = N'USA'
ORDER BY region
OFFSET 0 ROWS;
GO

#### View Options (ENCRYPTION, SCHEMABINDING, CHECK OPTION)

- Views can have options like `ENCRYPTION` to hide their definition, 
- `SCHEMABINDING` to bind to the schema of underlying tables, 
- and `CHECK OPTION` to enforce data integrity (it prevents modifications).

In [ ]:
-- Without Encryption

CREATE OR ALTER VIEW Sales.USACusts
AS
SELECT
  custid, companyname, contactname, contacttitle, address,
  city, region, postalcode, country, phone, fax
FROM Sales.Customers
WHERE country = N'USA';
GO

In [ ]:
SELECT OBJECT_DEFINITION(OBJECT_ID('Sales.USACusts'));

In [ ]:
-- ENCRYPTION option

-- DROP VIEW IF EXISTS Sales.USACusts;
-- GO

CREATE OR ALTER VIEW Sales.USACusts WITH ENCRYPTION
AS
SELECT
  custid, companyname, contactname, contacttitle, address,
  city, region, postalcode, country, phone, fax
FROM Sales.Customers
WHERE country = N'USA';
GO


In [ ]:
-- SCHEMABINDING option

-- DROP VIEW IF EXISTS Sales.USACusts;
-- GO

CREATE OR ALTER VIEW Sales.USACusts WITH SCHEMABINDING
AS
SELECT
  custid, companyname, contactname, contacttitle, address,
  city, region, postalcode, country, phone, fax
FROM Sales.Customers
WHERE country = N'USA';
GO

In [ ]:
-- CHECK OPTION

CREATE OR ALTER VIEW Sales.USACusts WITH SCHEMABINDING
AS
SELECT
  custid, companyname, contactname, contacttitle, address,
  city, region, postalcode, country, phone, fax
FROM Sales.Customers
WHERE country = N'USA'
WITH CHECK OPTION;
GO

In [ ]:
-- Error if CHECK OPTION

/*

INSERT INTO Sales.USACusts(
  companyname, contactname, contacttitle, address,
  city, region, postalcode, country, phone, fax)
 VALUES(
  N'Customer FGHIJ', N'Contact FGHIJ', N'Title FGHIJ', N'Address FGHIJ',
  N'London', NULL, N'12345', N'UK', N'012-3456789', N'012-3456789');

-- When Done:
DELETE FROM Sales.Customers
WHERE custid > 91;

DROP VIEW IF EXISTS Sales.USACusts;

*/

#### Inline Table-Valued Functions (TVFs)

- An inline table-valued function is a function that returns a table.
- Inline TVFs are _reusable table expressions_ that support input parameters.
- It is similar to a view but with parameters.

The following code creates an inline TVF called _GetCustOrders_ in the TSQLV6 database:

In [ ]:
USE TSQLV6;
GO
CREATE OR ALTER FUNCTION dbo.GetCustOrders
 (@cid AS INT) RETURNS TABLE -- mandatory in the header
AS
RETURN -- mandatory before the inner query
 SELECT orderid, custid, empid, orderdate, requireddate,
 shippeddate, shipperid, freight, shipname, shipaddress, shipcity,
 shipregion, shippostalcode, shipcountry
 FROM Sales.Orders
 WHERE custid = @cid;
GO

In [ ]:
-- Retrieve results

SELECT O.orderid, O.custid, OD.productid, OD.qty
FROM dbo.GetCustOrders(1) AS O
    INNER JOIN Sales.OrderDetails AS OD
        ON O.orderid = OD.orderid;

-- DROP FUNCTION IF EXISTS dbo.GetCustOrders;

#### The APPLY Operator

The APPLY operator allows you _to join a table to a table-valued function_ or derived table that references the outer query.

_**Note**_ _APPLY_ isn’t standard; the standard counterpart is called _LATERAL_, but the standard form wasn’t implemented in SQL Server.

- The _APPLY_ operator operates on two input tables; we can refer to them as the “left” and “right” tables. 
The right table is typically a derived table or a TVF.
- The CROSS APPLY operator implements one logical-query processing phase—it applies the right table to each row from the left table and produces a result table with the unified result sets

In [ ]:
-- CROSS JOIN

SELECT S.shipperid, E.empid
FROM Sales.Shippers AS S
 CROSS JOIN HR.Employees AS E;

-- Remember that a join treats its two inputs as a set, 
-- and therefore there’s no order between them.

In [ ]:
-- CROSS APPLY

SELECT S.shipperid, E.empid
FROM Sales.Shippers AS S
 CROSS APPLY HR.Employees AS E;

-- With APPLY, the left side is evaluated first, 
-- and the right side is evaluated per row from the left. 
-- So the right side can have references to 
-- elements from the left.

-- Those references are essentially CORRELATIONs.

Applying the CROSS APPLY operator to return the <u>three most recent orders for each customer</u>:

In [ ]:
SELECT C.custid, A.orderid, A.orderdate
FROM Sales.Customers AS C
 CROSS APPLY
 (SELECT TOP (3) orderid, empid, orderdate, requireddate
 FROM Sales.Orders AS O
 WHERE O.custid = C.custid
 ORDER BY orderdate DESC, orderid DESC) AS A;

Remember that you can use the standard `OFFSET-FETCH` option instead of `TOP`, like this:

In [ ]:
SELECT C.custid, A.orderid, A.orderdate
FROM Sales.Customers AS C
 CROSS APPLY
    (SELECT orderid, empid, orderdate, requireddate
    FROM Sales.Orders AS O
    WHERE O.custid = C.custid
    ORDER BY orderdate DESC, orderid DESC
    OFFSET 0 ROWS FETCH NEXT 3 ROWS ONLY) AS A;